# The Recurrent Neural Network

## Learning objectives

1. Understand the principles behind the creation of the recurrent neural network
2. Obtain intuition about difficulties training RNNs, namely: vanishing/exploding gradients and long-term dependencies
3. Obtain intuition about mechanics of backpropagation through time BPTT
4. Develop a Long Short-Term memory implementation in Keras 
5. Learn about uses and limitations of RNNs from a cognitive science perspective

## Historical and theoretical background

The poet Derlmore Schartz once wrote: **"...time is the fire in which we burn"**. We can't scape time. Time is embedded in every human thought and action. Yet, so far, we have been oblivious to the role of time in neural network modeling. Indeed, in all models we have examined so far we have implicitly assumed that **data is "perceived" all at once**, although there are countless examples where time is a critical consideration: movement, speech production, planning, decision-making, etc. We also have implicitly assumed that **past-states have no influence in future-states**. This is, the input pattern at time-step $t-1$ has no influence in the output of time-step $t-0$, or $t+1$, or any subsequent outcome for that matter. In probabilistic jargon, this equals to assume that each sample is drawn independently from each other. We know in many scenarios this is simply not true: when giving a talk, my next utterance will depend upon my past utterances; when running, my last stride will condition my next stride, and so on. You can imagine endless examples.

Multilayer Perceptrons and Convolutional Networks, in principle, can be used to approach problems where time and sequences are a consideration (for instance [Cui et al, 2016](https://arxiv.org/pdf/1603.06995.pdf)). Nevertheless, introducing time considerations in such architectures is cumbersome, and better architectures have been envisioned. In particular, **Recurrent Neural Networks (RNNs)** are the modern standard to deal with **time-dependent** and/or **sequence-dependent** problems. This type of networks are "recurrent" in the sense that they can **revisit or reuse past states as inputs to predict the next or future states**. To put it plainly, they have **memory**. Indeed, memory is what allow us to incorporate our past thoughts and behaviors into our future thoughts and behaviors.

### Hopfield Network

One of earliest examples of networks incorporating "recurrences" was the so-called **Hopfield Network**, introduced in 1982 by [John Hopfield](https://en.wikipedia.org/wiki/John_Hopfield), at the time, a physicist at Caltech. Hopfield networks were important as they helped to reignite the interest in neural networks in the early '80s (along with backpropagation). In his 1982 paper, Hopfield wanted to address the fundamental question of **emergence** in cognitive systems: Can relatively stable cognitive phenomena, like memories, emerge from the collective action of large numbers of simple neurons? After all, such behavior was observed in other physicial systems like vortex patterns in fluid flow. Brains seemed like another promising candidate.

Hopfield networks are known as a type of **energy-based** (instead of error-based) network because their properties derive from a global energy-function (Raj, 2020). In resemblence to the McCulloch-Pitts neuron, Hopfield neurons are binary threshold units but with recurrent instead of feed-forward connections, where each unit is **bi-directionally connected** to each other, as shown in **Figure 1**. This means that each unit *receives* inputs and *sends* inputs to every other connected unit. A consequence of this architecture is that **weights values are symmetric**, such that weights *coming into* a unit are the same as the ones *coming out* of a unit. The value of each unit is determined by a linear function wrapped into a threshold function $T$, as $y_i = T(\sum w_{ji}y_j + b_i)$.

<center> Figure 1: Hopfield Network </center>

<img src="./images/rec-net/hopfield-net.svg">

The basic idea of Hopfield networks is that each configuration of binary-values $C$ in the network is associated with a **global energy value $-E$**. Here is a simplified picture of the training process: imagine you have a network with five neurons with a configuration of $C_1=(0, 1, 0, 1, 0)$. Now, imagine $C_1$ yields a global energy-value $E_1= 2$ (following the energy function formula). Your goal is to *minimize* $E$ by changing one element of the network $c_i$ at a time. By using the weight updating rule $\Delta w$, you can subsequently get a new configuration like $C_2=(1, 1, 0, 1, 0)$, as new weights will cause a change in the activation values $(0,1)$. If $C_2$ yields a *lower value of $E$*, let's say, $1.5$, you are moving in the right direction. If you keep iterating with new configurations the network will eventually "settle" into a **global energy minimun** (conditioned to the initial state of the network).

A fascinanting aspect of Hopfield networks, besides the introduction of recurrence, is that is closely based in neuroscience research about learning and memory, particularly Hebbian learning (Hebb, 1949). In fact, Hopfield (1982) proposed this model as a way to capture **memory formation and retrieval**. Basically, the idea is that the energy-minima of the network could represent the **formation of a memory**, which further give rise to a property known as **[content-addressable memory (CAM)](https://en.wikipedia.org/wiki/Content-addressable_memory)**. Here is the idea with a computer analogy: when you access information storaged in the random access memory of your computer (RAM), you give the "address" where the "memory" is located to retrieve it. CAM works the other way around: you give information about the **content** you are searching for, and the computer should retrieve the "memory". This is great because this works even when you have **partial or corrupted** information about the content, which is a much more **realistic depiction of how human memory works**. It is similar to doing a google search. Just think in how many times you have searched for lyrics with partial information, like "song with the beeeee bop ba bodda bope!".

Is important to highlight that the sequential adjustment of Hopfield networks is **not driven by error correction**. Actually, there isn't a "target" as in supervised-based neural networks. Hopfield networks are systems that "evolve" until they find an stable low-energy state. If you "perturb" such a system, the system will "re-evolve" towards its previous stable-state, similar to how those inflatable "Bop Bags" toys get back to their initial position no matter how hard you punch them. It is almost like the system "remembers" its previous stable-state (in't?). This ability to "return" to a previous stable-state after perturbation is why they serve as models of memory.

### Elman Network

Although Hopfield networks where innovative and fascinating models, the first successful example of a recurrent network trained with backpropagation was introduced by [Jeffrey Elman](https://en.wikipedia.org/wiki/Jeffrey_Elman), the so-called **Elman Network** (Elman, 1990). Elman was a cognitive scientist at UC San Diego at the time, part of the group of researchers that published the famous PDP book.

In 1990, Elman published "Finding Structure in Time", an highly influential work for both cognitive science and machine learning (particularly natural language processing). Elman was concerned with the problem of representing "time" or "sequences" in neural networks. In his view, you could take either a "explicit" approach or an "implicit" approach. The **explicit** approach represents time **spacially**. Consider a vector $x = [x_1,x_2 \cdots, x_n]$, where element $x_1$ represents the first value of a sequence, $x_2$ the second element, and $x_n$ the last element. Hence, the spacial location in $\bf{x}$ is indicating the temporal location of a value. You can think about elements of $\bf{x}$ as sequences of words or actions, one after the other, for instance: $x^1=[Sound, of, the, funky, drummer]$ is a sequence of length five. Elman saw **several drawbacks** to this approach. First, although $\bf{x}$ is a sequence, the network still needs to represent the sequence all at once as an input, this is, a network would need five input neurons to process $x^1$. Second, imposes a rigid limit on the durations of pattern, in other words, the newtwork needs fixed number of elements for every input vector $\bf{x}$: a network with five input units, can't accomodate a sequence of lenght six. True, you could start with a six input network, but then shorter sequences would be misrepresented, since mistmatched units would receive zero input. This is a problem for most domains where sequences have variable duation. Finally, it can't easily distinguish **relative** temporal position from **absolute** temporal position. Consider the sequence $s = [1, 1]$ and a vector input lenght of four bits. Such a sequence can be presented in at least three variations:

$$
x_1 = [0, 1, 1, 0]\\
x_2 = [0, 0, 1, 1]\\
x_3 = [1, 1, 0, 0]
$$

Here, $\bf{x_1}$, $\bf{x_2}$, and $\bf{x_3}$ are instances of $\bf{s}$ but spacially displaced in the input vector. Geometrically, those three vectors are very different from each other (you can compute similarity measures to put a number on that), although represent the same instance. Even though you can train a neural net to learn those three patterns are associated with the same target, their inherent disimilarity will hinder the network ability to generalize the learned association. It is like training network to learn that blue, yellow, and red are associated with the same target "1": the network will learn the association, but it's unlikely to make sense outside the training set.

The **implicit** approach represents time by **its effect in intermediate computations**. To do this, Elman added a **contex unit** to save past computations and incorporate those in future computations. In short, **memory**. Elman based his approach in the work of [Michael I. Jordan](https://people.eecs.berkeley.edu/~jordan/) on serial processing (1986). Jordan's network implement recurrent connections from the network output $\hat{y}$ to its hidden units $h$, via a "memory unit" $\mu$ (equivalent to Elman's "context unit") as depicted in **Figure 2**. In short, the the memory unit keeps a running average of **all past outputs**: this is how the entire past history is implicitly accounted for on each new computation. There is no learning in the memory unit, which means the weights are fixed to $1$.

<center> Figure 2: Jordan Network </center>

<img src="./images/rec-net/jordan-net.svg">

**Note**: Jordan's network diagrams exemplifies the two ways in which recurrent nets are usually represented. On the left, the **compact format** depicts the network structure as a circuit. On the right, the **unfolded representation** incorporates the notion of time-steps calculations. The unfolded representation also illustrate how a recurrent network can be constructed in a pure feed-forward fashion, with as many layers as time-steps in your sequence. One key consideration is that the weights will be identical on each time-step (or layer). Keep this unfolded representation in mind as will become important later.

Elman's innovation was twofold: **recurrent conections between hidden units and memory** (contex) units, and **trainable parameters from the memory units to the hidden units**. Memory units now have to "remember" the past state of hidden units, which means that instead of keeping a running average, they "clone" the value at the previous time-step $t-1$. Memory units also have to learn useful representations (weights) for encoding temporal properties of the sequential input.  **Figure 3** summarizes Elman's network in compact and unfolded fashion.

<center> Figure 3: Elman Network </center>

<img src="./images/rec-net/elman-net.svg">

**Note**: there is something curious about Elman's architecture. What it is the point of "cloning" $h$ into $c$ at each time-step? You could bypass $c$ altogether by sending the value of $h_t$ straight into $h_{t+1}$, wich yield mathematically identical results. The most likely explanation for this was that Elman starting point was Jordan's network, which had a separated memory unit. Regardless, keep in mind we don't need $c$ units to design a functionally identical network. 

Elman performed multiple experiments with this architecture demostrating it was capable to solve multiple problems with a sequential structure: a temporal version of the XOR problem; learning the structure (i.e., vowels and consonants sequential order) in sequences of letters; discovering the notion of "word"; and even learning complex lexical classes like word order in short sentences. Let's briefly explore the temporal XOR solution as an exemplar. **Table 1** shows the XOR problem:

**Table 1**: Truth Table For XOR Function

| $x_1$ | $x_2$ | $y$ |
|---|---|--------|
| 0 | 0 | 0      |
| 0 | 1 | 1      |
| 1 | 0 | 1      |
| 1 | 1 | 0      |

Here is a way to transform the XOR problem into a sequence. Consider the following vector: 

$$
s= [1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1,...]
$$

In $\bf{s}$, the first and second elements, $s_1$ and $s_2$, represent $x_1$ and $x_2$ inputs of **Table 1**, whereas the third element, $s_3$, represents the corresponding output $y$. This pattern repeats until the end of the sequence $s$ as shown in **Figure 4**.

<center> Figure 4: Temporal XOR </center>

<img src="./images/rec-net/temporal-xor.svg">

Elman trained his network with a 3,000 elements sequence for 600 iterations over the entire dataset, on the task of predicting the next item $s_{t+1}$ of the sequence $s$, meaning that he fed inputs to the network **one by one**. He showed that **error pattern** followed a predictable trend: the mean squared error was **lower every 3 outputs**, and higher in between, meaning the network learned to predict the third element in the sequence, as shown in **Chart 1** (the numbers are made up, but the pattern is the same found by Elman (1990)).

In [1]:
import numpy as np
import pandas as pd
import altair as alt

In [2]:
s = pd.DataFrame({"MSE": [0.35, 0.15, 0.30, 0.27, 0.14, 0.40, 0.35, 0.12, 0.36, 0.31, 0.15, 0.32],
                  "cycle": np.arange(1, 13)})
alt.Chart(s).mark_line().encode(x="cycle", y="MSE").properties(title='Chart 1')

alt.Chart(...)

An inmediate advantage of this approach is the network can take **inputs of any lenght**, withouth having to alter the networ architecture at all.

In the same paper, Elman showed that the **internal (hidden) representations** learned by the network grouped into meaningful categories, this is, **semantically similar words group together** when analyzed with [hierarchical clustering](https://en.wikipedia.org/wiki/Hierarchical_clustering). This was remarkable as demostrated the utility of RNNs as model of cognition in sequence-based problems. 

## Interlude: vanishing and exploding gradients in RNNs

Turns out, training recurrent neural networks is hard. Considerably harder than multilayer-perceptrons. When faced with the task of training **very deep networks**, like RNNs, the gradients have the impolite tendency of either (1) **vanishing**, or (2) **exploding** (Bengio et all, 1994; Pascanu et all, 2012). Recall that RNNs can be unfolded so that recurrent connections follow pure feed-forward computations. This unrolled RNN will have as many layers as elements in the sequence. Thus, a sequence of 50 words will be unrolled as a RNN of 50 layers (taking word as a unit). 

Concretely, the **vanishing gradient problem** will make really hard to learn **long-term dependencies** in sequences. Let's say you have a collection of poems, where the last sentence makes reference to the first one. Such a dependency will be hard to learn for a deep RNN where gradients vanish as we move backward in the network. The **exploding gradient problem** will completely derail the learning process. In very deep networks this is often a problem because more layers amplify the effect of large gradients, compounding into very large updates to the network weights, to the point values completely blow up.        

Here is the intuition for the **mechanics of gradient vanishing**: when gradients *begin small*, as you move backwards through the network computing gradients, they will get even smaller as you get closer to the input layer. Consequently, when doing the weight update based on such gradients, the weights closer to the output layer will obtain larger updates than weights closer to the input layer. This means that the weights closer to the input layer will hardly change at all, whereas the weights closer to the ouput layer will change a lot. This is a serious problem when **earlier layers matter for prediction**: they will keep propagating more or less the same signal forward because no learning (i.e., weight updates) will happen, which may significantly hinder the network performance.  

Here is the intuition for the **mechanics of gradient explotion**: when gradients *begin large*, as you move backwards through the network computing gradients, they will get even larger as you get closer to the input layer. Consequently, when doing the weight update based on such gradients, the weights closer to the input layer will obtain larger updates than weights closer to the output layer. Learning can go wrong really fast. Recall that the signal propagated by each layer is the outcome of taking the product between the previous hidden-state and the current hidden-state. If the weights in earlier layers get really large, they will forward-propagate larger and larger signals on each iteration, and the predicted output values will spiral-up out of control, making the error $y-\hat{y}$ so large that the network will be unable to learn at all. In fact, your computer will "overflow" quickly as it would unable to represent numbers that big. Very dramatic.  

The mathematics of gradient vanishing and explotion gets complicated quickly. If you want to delve into the mathematics see [Bengio et all (1994)](http://ai.dinfo.unifi.it/paolo/ps/tnn-94-gradient.pdf), [Pascanu et all (2012)](https://arxiv.org/abs/1211.5063), and [Philipp et all (2017)](https://arxiv.org/abs/1712.05577). 

For our purposes, I'll give you a simplified numerical example for intuition. Consider the task of predicting a vector $y = \begin{bmatrix} 1 & 1 \end{bmatrix}$, from inputs $x = \begin{bmatrix} 1 & 1 \end{bmatrix}$, with a multilayer-perceptron with 5 hidden layers and tanh activation functions. We have two cases:

- the weight matrix $W_l$ is initialized to large values $w_{ij} = 2$
- the weight matrix $W_s$ is initialized to small values $w_{ij} = 0.02$

Now, let's compute a single forward-propagation pass:

In [3]:
import numpy as np

In [4]:
x = np.array([[1],[1]])
W_l = np.array([[2, 2],
                [2, 2]])

h1 = np.tanh(W_l @ x)
h2 = np.tanh(W_l @ h1)
h3 = np.tanh(W_l @ h2)
h4 = np.tanh(W_l @ h3)
h5 = np.tanh(W_l @ h4)
y_hat = (W_l @ h5)
print(f'output for large initial weights: \n {y_hat}')

output for large initial weights: 
 [[3.99730269]
 [3.99730269]]


In [5]:
x = np.array([[1],[1]])
W_s = np.array([[0.02, 0.02],
                [0.02, 0.02]])

h1 = np.tanh(W_s @ x)
h2 = np.tanh(W_s @ h1)
h3 = np.tanh(W_s @ h2)
h4 = np.tanh(W_s @ h3)
h5 = np.tanh(W_s @ h4)
y_hat = (W_s @ h5)
print(f'output for small initial weights: \n {y_hat}')

output for small initial weights: 
 [[4.09381337e-09]
 [4.09381337e-09]]


We see that for $W_l$ the output $\hat{y}\approx4$, whereas for $W_s$ the output $\hat{y} \approx 0$. Why does this matter? We haven't done the gradient computation but you can probably anticipate what it's goin to happen: for the $W_l$ case, the gradient update is going to be very large, and for the $W_s$ very small. If you keep cycling through forward and backward passes these problems will become worse, leading to gradient explotion and vanishing respectively.

### Long Short-Term Memory Network

Several challenges difficulted progress in RNN in the early '90s (Hochreiter & Schmidhuber, 1997; Pascanu et all, 2012). In addition to vanishing and exploding gradients, we have the fact that the **forward computation is slow**, as RNNs can't compute in parallel: to preserve the time-dependencies through the layers, each layer has to be computed sequentially, which naturally takes more time. Elman networks proved to be effective at solving relatively simple problems, but as the sequences scaled in size and complexity, this type of network struggle. 

Several approaches where proposed in the '90s to address the aforementioned issues like time-delay neural networks (Lang et al, 1990), simulated annealing (Bengio et al., 1994), and others. The architecture that really moved the field forward was the so-called **Long Short-Term Memory (LSTM) Network**, introduced by [Sepp Hochreiter](https://en.wikipedia.org/wiki/Sepp_Hochreiter) and [Jurgen Schmidhuber](https://en.wikipedia.org/wiki/J%C3%BCrgen_Schmidhuber) in 1997. As the name sugguest, the defining characteristic of LSTMs is the addition of units combining both short-memory and long-memory capabilities. 

In LSTMs, instead of having a simple memory unit "cloning" values from the hidden unit as in Elman networks, we have a (1) **cell unit** (a.k.a., memory unit) which effectively acts as a long-term memory storage, and (2) a **hidden-state** which acts as a memory controller. These two elements are integrated as a circuit of logic gates controlling the flow of information at each time-step. Understanding the notation is crucial here, which is depicted in **Figure 5**.

<center> Figure 5: LSTM architecture </center>

<img src="./images/rec-net/lstm-unit.svg">

In LSTMs $x_t$, $h_t$, and $c_t$ represent vectors of values. Lightish-pink circles represent element-wise operations, and darkish-pink boxes are fully-connected layers with trainable weights. The top part of the diagram acts as a **memory storage**, whereas the bottom part has a double role: (1) passing the hidden-state information from the previos time-step $t-1$ to the next time step $t$, and (2) to regulate the **influx** of information from $x_t$ and $h_{t-1}$ **into** the memory storage, and the **outflux** of information **from** the memory storage into the next hidden state $h-t$. The second role is the core idea behind LSTM. You can think about it as making **three decisions** at each time-step:

1. **Is the *old information* $c_{t-1}$ worth to keep in my memory storage $c_t$?** If so, let the information pass, otherwise, "forget" such information. This is controlled by the *forget gate*.    
2. **Is this *new information* (inputs) worth to be saved into my memory storage $c_t$?** If so, let information flow into $c_t$. This is controlled by the *input gate* and the *candidate memory cell*. 
3. **What elements of the information saved in my memory storage $c_t$ are relevant for the computation of the next hidden-state $h_t$?** Select them from $c_t$, combine them new hidden-state output, and let the pass into the next hidden-state $h_t$. This is controlled by the *output gate* and the *tanh* function. 

Decisions 1 and 2 will determine the information that keeps flowing through the memory storage at the top. Decision 3 will determine the information that flows to the next hidde state at the bottom. The conjunction of these decisions sometimes is called "memory block".

<center> Figure 6: LSTM as a sequence of decisions </center>

<img src="./images/rec-net/lstm-choices.svg">

To put LSTMs in context, imagine the following simplified scenerio: we are trying to **predict the next word in a sequence**. Let's say, squences are about sports. From past sequences, we saved in the memory block the type of sport: "soccer". For the current sequence, we receive a phrase like "A basketball player...". In such a case, we first want to "forget" the previous type of sport "soccer" (*decision 1*) by multplying $c_{t-1} \odot f_t$. Next, we want to "update" memory with the new type of sport, "basketball" (*decision 2*), by adding $c_t = (c_{t-1} \odot f_t) + (i_t \odot \tilde{c_t})$. Finally, we want to output (*decision 3*) a verb relevant for "A basketball player...", like "shoot" or "dunk" by $\hat{y_t} = softmax(W_{hz}h_t + b_z)$.

LSTMs long-term memory capabilities make them good at capturing long-term dependencies. The memory cell effectivelly counteract the vanishing gradient problem at preserving information as long the forget gate does not "erase" past information (Graves, 2012). All the above make LSTMs succesful in practical applications in sequence-modeling (see a list [here](https://en.wikipedia.org/wiki/Long_short-term_memory#Applications)). For instance, when you use [Google's Voice Transcription](https://ai.googleblog.com/2015/08/the-neural-networks-behind-google-voice.html) services there is a RNN doing the hard work of recognizing your voice.

### RNNs and cognition

As with Convolutional Neural Networks, researchers utilizing RNN for approaching sequential problems like natural languague processing (NLP) or time-series prediction, do not *necessarily* care about (althouh some might) how good of a model of cognition and brain-activity are RNNs. What they really care is about solving problems like translation, speech recognition, and stock market prediction, and many advances in the field come from pursuing such goals. Still, RNN have many **desirable traits as model of neuro-cognitive activity**, and have been prolifically used to **model several aspects of human cognition and behavior**: child behavior in a object permanence tasks (Munakata et all, 1997); knowledge-intensive text-comprehension (St. John, 1992); processing in quasi-regular domains, like English word reading (Plaut et al., 1996); human performance in processing recursive language structures (Christiansen & Chater, 1999); human sequential action (Botvinick & Plaut, 2004); movement patterns in typical and atypical developing children (Muñoz-Organero et al., 2019). And many others. Neuroscientist have used RNNs to model a wide variety of aspects as well (for reviews see Barak, 2017, Güçlü & van Gerven, 2017, Jarne & Laje, 2019). Overall, RNN have demostrated to be a productive tool for modeling cognitive and brain function, in distributed representations paradigm.

## Mathematical formalization

There are two mathematically complex issues with RNNs: (1) computing hidden-states, and (2) backpropagation. The rest are common operations found in multilayer-perceptrons. LSTMs and its many variants are the facto standard when modeling any kind of sequential problem. Elman networks can be seen as a simplified version of a LSTM, so I'll focus my attention on LSTMs for the most part. My exposition is based in a combination of sources that you may want to review for extended explanations (Bengio et al., 1994; Hochreiter & Schmidhuber, 1997; Graves, 2012; Chen, 2016; Zhang et al., 2020).

The LSTM architecture can be desribed by: 

**Forward pass**:
- non-linear forget function
- non-linear input function 
- non-linear candidate-memory function
- non-linear output function
- memory cell function
- non-linear hidden-state function 
- softmax function (output)

**Backward pass**:
- Cost-function
- Learning procedure (backpropagation)

Following the indices for each function requires some definitions. I'll assume we have $h$ hidden units, training sequences of size $n$, and $d$ input units. 

$$
\begin{align}
\text{input-units} &= x_i \in \mathbb{R}^d \\ 
\text{training-sequence} &= s_i \in \mathbb{R}^n \\
\text{output-class} &= y_i \in \mathbb{R}^k \\
\text{Input-layer} &= X_t \in \mathbb{R}^{n\times d} \\
\text{hidden-layer} &= H_t \in \mathbb{R}^{n\times h}
\end{align}
$$

## Forget function

The **forget function** is a sigmoidal mapping combining three elements: input vector $x_t$, past hidden-state $h_{t-1}$, and a bias term $b_f$. We didn't mentioned the bias before, but it is the same bias that all neural networks incorporate, one for each unit in $f$. More formally:

$$
f_t = \sigma(W_{xf}x_t + W_{hf}h_{t-1} + b_f)
$$ 

Each matrix $W$ has dimensionality equal to (number of incoming units, number for connected units). For example, $W_{xf}$ referes to $W_{input-units, forget-units}$. Keep this in mind to read the indices of the $W$ matrices for subsequent definitions.

Here is an important insight: What would it happen if $f_t = 0$? If you look at the diagram in **Figure 6**, $f_t$ performs an elementwise multiplication of each element in $c_{t-1}$, meaning that every value would be reduced to $0$. In short, the network would completely "forget" past states. Naturally, if $f_t = 1$, the network would keep its memory intact. 

## Input function and Candidate memory function

The **input function** is a sigmoidal mapping combining three elements: input vector $x_t$, past hidden-state $h_{t-1}$, and a bias term $b_f$. It's defined as:

$$
i_t = \sigma(W_{xi}x_t + W_{hi}h_{t-1} + b_i)
$$ 

The **candidate memory function** is an hyperbolic tanget function combining the same elements that $i_t$. It's defined as:

$$
\tilde{c}_t = tanh(W_{xc}x_t + W_{hc}h_{t-1} + b_c)
$$ 

Both functions are combined to update the memory cell.

## Output function

The **output function** is a sigmoidal mapping combining three elements: input vector $x_t$, past hidden-state $h_{t-1}$, and a bias term $b_f$. Is defined as:

$$
o_t = \sigma(W_{xo}x_t + W_{ho}h_{t-1} + b_o)
$$ 

## Memory cell function

The **memory cell function** (what I've been calling "memory storage" for conceptual clarity), combines the effect of the forget function, input function, and candidate memory function. It's defined as:

$$
c_t = (c_{t-1} \odot f_t) + (i_t \odot \tilde{c_t})
$$

Where $\odot$ implies an elementwise multiplication (instead of the usual dot product). This expands to:

$$
c_t = (c_{t-1} \odot \sigma(W_{xf}x_t + W_{hf}h_{t-1} + b_f)) + (\sigma(W_{xi}x_t + W_{hi}h_{t-1} + b_i) \odot tanh(W_{xc}x_t + W_{hc}h_{t-1} + b_c))
$$


## Hidden state function

The next **hidden state function** combines the effect of the output function and the contents of the memory cell scaled by a tanh function. It is defined as:

$$
h_t = O_t \odot tanh(c_t) 
$$

## Output function

The output function will depend upon the problem to be approached. For our our purposes, we will assume a multi-class problem, for which the **softmax function** is appropiated. For this, we first pass the hidden-state by a linear function, and then the softmax as:


$$
\begin{align}
z_t &= (W_{hz}h_t + b_z)\\
\hat{y}_t &= softmax(z_t) = \frac{e^{z_t}}{\sum_{j=1}^K e^{z_j}} 
\end{align}
$$

The softmax computes the exponent for each $z_t$ and then normalize by dividing by the sum of every output value exponentiated. In this manner, the output of the softmax can be interpreted as the likelihood value $p$.

## Cost function 

As with the output function, the cost function will depend upon the problem. For regression problems, the Mean-Squared Error can be used. For our purposes (classification), the cross-entropy function is appropiated. It's defined as:

$$
E_i = - \sum_t y_ilog(p_i)
$$

Where $y_i$ is the true label for the $ith$ output unit, and $log(p_i)$ is the log of the softmax value for the $ith$ output unit. The summation indicates we need to aggregate the cost at each time-step.

### Learning procedure: Backpropagation Through Time (BPTT)

Originally, Hochreiter and Schmidhuber (1997) trained LSTMs with a combination of approximate gradient descent computed with a combination of real-time recurrent learning and backpropagation through time (BPTT). Nevertheless, LSTM can be trained with pure backpropagation. Following Graves (2012), I'll only describe BTT because is more accurate,  easier to debug, and to describe.

**Note**: we call it **backpropagation through time** because of the sequential time-dependent structure of RNNs. Recall that each layer represents a time-step, and forward propagation happens in sequence, one layer computed after the other. Hence, when we backpropagate, we do the same but backwards (i.e., "through time").

<center> Figure 7: Three-layer simplified RNN </center>

<img src="./images/rec-net/simple-rnn.svg">

I reviewed backpropagation for a simple multilayer perceptron [here](https://com-cog-book.github.io/com-cog-book/features/multilayer-perceptron.html#Backpropagation-algorithm). Nevertheless, I'll sketch BPTT for the simplest case as shown in **Figure 7**, this is, with a genereic non-linear hidden-layer similar to Elman network withouth "context units" (some like to call it "vanilla" RNN, which I avoid because I beleive is derogatory against vanilla). This exercise will allow us to review backpropagation and to understand how it differs from BPTT. We begin by defining a simplified RNN as: 

$$
\begin{align}
z_t &= W_{hz}h_t + b_z\\
h_t &= \sigma(W_{hh}h_{t-1} + W_{xh}x_t+b_h)
\end{align}
$$

Where $h_t$ and $z_t$ indicates a hidden-state (or layer) and  the output respectively. Therefore, **we have to compute gradients w.r.t. five sets of weights**: $\{W_{hz}, W_{hh}, W_{xh}, b_z, b_h\}$.



First, consider the error derivatives w.r.t. $W_{hz}$ at time $t$, the weight matrix for the linear function at the output layer. Recall that $W_{hz}$ is shared across all time-steps, hence, we can compute the gradients at each time step and then take the sum as:

$$
\frac{\partial{E}}{\partial{W_{hz}}} = \sum_t\frac{\partial{E}}{\partial{z_t}} \frac{\partial{z_t}}{\partial{W_{hz}}}
$$

Same for the bias term:

$$
\frac{\partial{E}}{\partial{b_z}} = \sum_t\frac{\partial{E}}{\partial{z_t}} \frac{\partial{z_t}}{\partial{b_z}}
$$


That part is straightforward. The issue arises when we try to compute the gradients w.r.t. the wights $W_{hh}$ in the hidden layer. Consider a three layer RNN (i.e., unfolded over three time-steps). In such a case, we have: 

- $E_3$ depens on $z_3$
- $z_3$ depends on $h_3$
- $h_3$ depens on $h_2$
- $h_2$ depens on $h_1$
- $h_1$ depens on $h_0$, where $h_0$ is a random starting state.

Now, we have that $E_3$ w.r.t to $h_3$ becomes:

$$
\frac{\partial{E_3}}{\partial{W_{hh}}} = 
\frac{\partial{E_3}}{\partial{z_3}}
\frac{\partial{z_3}}{\partial{h_3}}
\frac{\partial{h_3}}{\partial{W_{hh}}}
$$

The issue here is that $h_3$ depends on $h_2$, since according to our definition, the $W_{hh}$ is multiplied by $h_{t-1}$, meaning **we can't compute $\frac{\partial{h_3}}{\partial{W_{hh}}}$ directly**. Othewise, we would be treating $h_2$ as a constant, which is incorrect: is a function. What we need to do is to **compute the gradients separately**: the direct contribution of ${W_{hh}}$ on $E$ and the indirect contribution via $h_2$. Following the rules of calculus in multiple variables, we compute them independently and add them up together as:

$$
\frac{\partial{E_3}}{\partial{W_{hh}}} = 
\frac{\partial{E_3}}{\partial{z_3}}
\frac{\partial{z_3}}{\partial{h_3}}
\frac{\partial{h_3}}{\partial{W_{hh}}}+
\frac{\partial{E_3}}{\partial{z_3}}
\frac{\partial{z_3}}{\partial{h_3}}
\frac{\partial{h_3}}{\partial{h_2}}
\frac{\partial{h_2}}{\partial{W_{hh}}}
$$

Again, we have that we can't compute $\frac{\partial{h_2}}{\partial{W_{hh}}}$ directly. Following the same procedure, we have that our full expression becomes:

$$
\frac{\partial{E_3}}{\partial{W_{hh}}} = 
\frac{\partial{E_3}}{\partial{z_3}}
\frac{\partial{z_3}}{\partial{h_3}}
\frac{\partial{h_3}}{\partial{W_{hh}}}+
\frac{\partial{E_3}}{\partial{z_3}}
\frac{\partial{z_3}}{\partial{h_3}}
\frac{\partial{h_3}}{\partial{h_2}}
\frac{\partial{h_2}}{\partial{W_{hh}}}+
\frac{\partial{E_3}}{\partial{z_3}}
\frac{\partial{z_3}}{\partial{h_3}}
\frac{\partial{h_3}}{\partial{h_2}}
\frac{\partial{h_2}}{\partial{h_1}}
\frac{\partial{h_1}}{\partial{W_{hh}}}
$$

Essentially, this means that we compute and add the contribution of $W_{hh}$ to $E$ at each time-step. The expression for $b_h$ is the same:

$$
\frac{\partial{E_3}}{\partial{b_h}} = 
\frac{\partial{E_3}}{\partial{z_3}}
\frac{\partial{z_3}}{\partial{h_3}}
\frac{\partial{h_3}}{\partial{b_h}}+
\frac{\partial{E_3}}{\partial{z_3}}
\frac{\partial{z_3}}{\partial{h_3}}
\frac{\partial{h_3}}{\partial{h_2}}
\frac{\partial{h_2}}{\partial{b_h}}+
\frac{\partial{E_3}}{\partial{z_3}}
\frac{\partial{z_3}}{\partial{h_3}}
\frac{\partial{h_3}}{\partial{h_2}}
\frac{\partial{h_2}}{\partial{h_1}}
\frac{\partial{h_1}}{\partial{b_h}}
$$

Finally, we need to compute the gradients w.r.t. $W_{xh}$. Here, again, we have to add the contributions of $W_{xh}$ via $h_3$, $h_2$, and $h_1$: 

$$
\frac{\partial{E_3}}{\partial{W_{xh}}} = 
\frac{\partial{E_3}}{\partial{z_3}}
\frac{\partial{z_3}}{\partial{h_3}}
\frac{\partial{h_3}}{\partial{W_{xh}}}+
\frac{\partial{E_3}}{\partial{z_3}}
\frac{\partial{z_3}}{\partial{h_3}}
\frac{\partial{h_3}}{\partial{h_2}}
\frac{\partial{h_2}}{\partial{W_{xh}}}+
\frac{\partial{E_3}}{\partial{z_3}}
\frac{\partial{z_3}}{\partial{h_3}}
\frac{\partial{h_3}}{\partial{h_2}}
\frac{\partial{h_2}}{\partial{h_1}}
\frac{\partial{h_1}}{\partial{W_{xh}}}
$$

That's for BPTT for a simple RNN. The math reviewed here generalize with minimal changes to more complex architectures as LSTMs. Actually, the only difference regarding LSTMs, is that we have more weights to differentiate for. Instead of a single generic $W_{hh}$, we have $W$ for all the gates: forget, input, output, and candidate cell. The rest remains the same. For a detailed derivation of BPTT fot the LSTM see [Graves (2012)](https://www.cs.toronto.edu/~graves/preprint.pdf) and [Chen (2016)](https://arxiv.org/abs/1610.02583).

## Interlude: Sequence-data representation

Working with sequence-data, like text or time-series, requieres to pre-process it in a manner that is "digestible" for RNNs. As with any neural network, RNN can't take raw text as an input, we need to **parse** text sequences and then **"map"** them into vectors of numbers. Here I'll briefly review these issues to provide enough context for our example applications. For an extended revision please refer to [Jurafsky and Martin (2019)](https://web.stanford.edu/~jurafsky/slp3/), [Goldberg (2015)](http://u.cs.biu.ac.il/~yogo/nnlp.pdf), [Chollet (2017)](https://www.manning.com/books/deep-learning-with-python), and [Zhang et al (2020)](https://d2l.ai/chapter_recurrent-neural-networks/index.html).

Parsing can be done in multiple manners, the most common being: 

- Using **word** as unit, which each word represented as a vector
- Using **character** as a unit, with each character represented as a vector
- Using **n-grams** of words or characters as unit, with each n-gram represented as a vector. N-grams are sets of words or characters of size "N" or less.

The process of parsing text into smaller units is called "tokenization", and each resulting unit is called a "token", the top pane in **Figure 8** display a sketch of the tokenization process.

<center> Figure 8: Tokenization </center>

<img src="./images/rec-net/text-pro.png">

Once a corpus of text has been parsed into tokens, we have to map such tokens into numerical vectors. Two common ways to do this are **one-hot encoding** approach and the **word embeddings** approach, as depicted in the bottom pane of **Figure 8**. We used one-hot encodings to transform the MNIST class-labels into vectors of numbers for classification in the [CovNets chapter](https://com-cog-book.github.io/com-cog-book/features/cov-net.html). In a one-hot encoding vector, each token is mapped into a *unique* vector of zeros and ones. The vector size is determined by the vocabullary size. For instance, for the set $x= \{"cat", "dog", "ferret"\}$, we could use a 3-dimensional one-hot encoding as:

$$
\text{cat}=
\begin{bmatrix}
1 \\
0 \\
0
\end{bmatrix},
\text{dog}=
\begin{bmatrix}
0 \\
1 \\
0
\end{bmatrix},
\text{ferret}=
\begin{bmatrix}
0 \\
0 \\
1
\end{bmatrix}
$$

One-hot encodings have the advantages of being straighforward to implement, and to provide an unique identifier for each token. Its main disadvantage is that tends to create really **sparse** and **high-dimensional** representations for large corpus of texts. For instance, if you tried a one-hot encoding for 50,000 tokens, you'd end up with a 50,000 by 50,000 dimensional matrix, which it may be unpractical for most tasks. 

**Word embeddings** represent text by mapping tokens into vectors of real-valued numbers instead of only zeros and ones. This significantly increments the representational capacity of vectors, reducing the required dimensionality for a given corpus of text compared to one-hot encodings. For instance, 50,000 tokens could be represented by as little as 2 or 3 vectors (although the representation may not be very good). Taking the same set $x$ as before, we could have a 2-dimensional word embedding like:

$$
\text{cat}=
\begin{bmatrix}
0.1 \\
0.8
\end{bmatrix},
\text{dog}=
\begin{bmatrix}
0.2 \\
1 
\end{bmatrix},
\text{ferret}=
\begin{bmatrix}
0.6 \\
0.2
\end{bmatrix}
$$

You may be wondering why to bother with one-hot encodings when word embeddings are much more space-efficient. The main issue with word-embedding is that **there isn't an obvious way to map tokens into vectors** as with one-hot encodings. For instance, you could assign tokens to vectors at random (assuming every token is assigned to a *unique* vector). The problem with such approach is that the semantic structure in the corpus is broken. Ideally, you want words of similar meaning mapped into similar vectors. We can preserve the semantic structure of a text corpus in the same manner than everything else in machine learning: by **learning from data**. There are two ways to do this:

- Learning the word embeddings **at the same time** you train the RNN.
- Utilizing **pretrained** word embeddings, this is, embeddings learned in a different task. This is a form of "transfer learning".

Learning word embeddings for your task is advisable as semantic relationships among words tend to be **context dependent**. For instance, "exploitation" in the context of mining is related to resource "extraction", hence relative neutral. But, "exploitation" in the context of labor rights is related to the idea of "abuse", hence a negative connotation. This is more critical when we are dealing with different languages. Nevertheless, learning embeddings for every task sometimes is impractical, either because your corpus is too "small" (i.e., not enough data to extract semantic relantionships), or too "large" (i.e., you don't have enough time and/or resources to learn the embeddings). Examples of freely accessible pretrained word embeddings are Google's [Word2vec](https://code.google.com/archive/p/word2vec/) and the [Global Vectors for Word Representation](https://nlp.stanford.edu/projects/glove/) (GloVe).

## Code implementation

As in previous chapters, I'll use Keras to implement both (a modified version of) the Elman Network for the XOR problem and a LSTM for text prediction.  

### Elman Network

By now, it may be clear to you that Elman networks are a simple RNN with two neurons, one for each input pattern, in the hidden-state. Originally, Elman trained his architecture with a truncated version of BPTT, meaning that only considered two time-steps for computing the gradients, $t$ and $t-1$. We will implement a modified version of Elman's architecture bypassing the "contex" unit (which does not alter the result at all) and utilizing BPTT instead of its truncated version. 

### Generating data 

Nowadays, we don't to generate the 3,000 bits sequence that Elman used in his original work. We can simply generate a single pair of training and testing for the XOR problem, and pass the training sequence (lenght two) as the inputs, and the expected outputs as the target. This is very much alike any classification task. An important caveat is that simpleRNN layes in Keras expect an input tensor of shape (number-samples, timesteps, number-input-features). In out case, this has to be: number-samples= 4,  timesteps=1, number-input-features=2. No separate encoding is necessary here because we are manually setting the input and output values to binary vector representations. Finally, we won't worry about training and testing sets for this example, which is way to simple for that (we will do that for the next example).

**Table 1**: Truth Table For XOR Function

| $x_1$ | $x_2$ | $y$ |
|---|---|--------|
| 0 | 0 | 0      |
| 0 | 1 | 1      |
| 1 | 0 | 1      |
| 1 | 1 | 0      |

In [6]:
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN
import altair as alt
from numpy.random import seed
seed(12345)

Using TensorFlow backend.


In [7]:
# features
X = np.array([[[0, 0, 1, 1]],
              [[0, 1, 0, 1]]]).T

# expected values
y = np.array([[0, 1, 1, 0]]).T

print(f'training data shape: {X.shape}')
print(f'targets data shape: {y.shape}')

training data shape: (4, 1, 2)
targets data shape: (4, 1)


### Elman network architecture in Keras

Defining a (modified) in Keras is extremely simple as shown below.

In [8]:
# Define a network as a linear stack of layers
model = Sequential()

# Add a recurrent layer with 2 units
model.add(SimpleRNN(2, input_shape=(1, 2)))

# Add the output layer with a sigmoid activation
model.add(Dense(1, activation='tanh'))


The model summary shows that our architecture yields 13 trainable parameters.

In [9]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 2)                 10        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 3         
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________


## Elman network Application: XOR classification

Next, we compile and fit our model. I'll utilie [Adadelta](https://github.com/keras-team/keras/blob/master/keras/optimizers.py#L376) (to avoid manually adjusting the learning rate) as the optimizer, and the Mean-Squared Error (as in Elman original work). I'll train the model for 15,000 epochs over the 4 samples dataset.

In [10]:
model.compile(optimizer='Adadelta', 
              loss='mean_squared_error', 
              metrics=['acc'])
history = model.fit(X, y,
                    epochs=5000,
                    verbose=0)

**Chart 2** shows the error curve (red, right axis), and the accuracy curve (blue, left axis) for each epoch.

In [11]:
alt.data_transformers.disable_max_rows()

loss = history.history['loss']
accuracy = history.history['acc']

df = pd.DataFrame({"accuracy":accuracy, "loss":loss, "time-step": np.arange(0, len(accuracy))})

base = alt.Chart(df).mark_line(color="blue").encode(x="time-step", y="accuracy")
loss = alt.Chart(df).mark_line(color="red").encode(x="time-step", y="loss")
(base  + loss).properties(title='Chart 2').resolve_scale(y='independent')

alt.LayerChart(...)

We see that accuracy goes to 100% in less than 1,000 epochs.

### LSTM

### Reading data from Keras

### Word embeddings with Keras

### LSTM architecture in Keras

## LSTM Applicatiion: text prediction 

## Limitations

## Conclusions

## References


- 3 gradients problem papers.
- RNN deep learning chapters
 
- Botvinick, M., & Plaut, D. C. (2004). Doing without schema hierarchies: A recurrent connectionist approach to normal and impaired routine sequential action. Psychological Review, 111(2), 395.
- Barak, O. (2017). Recurrent neural networks as versatile tools of neuroscience research. Current Opinion in Neurobiology, 46, 1–6. https://doi.org/10.1016/j.conb.2017.06.003
- Chen, G. (2016). A gentle tutorial of recurrent neural network with error backpropagation. arXiv preprint arXiv:1610.02583.
- Elman, J. L. (1990). Finding Structure in Time. Cognitive Science, 14(2), 179–211. https://doi.org/10.1207/s15516709cog1402_1
- François, C. (2017). 6. Deep Learning for text and sequences. Deep learning with Python. Manning.
- Christiansen, M. H., & Chater, N. (1999). Toward a connectionist model of recursion in human linguistic performance. Cognitive Science, 23(2), 157–205.
- Hebb, D. O. (1949). The organization of behavior: A neuropsychological theory. Psychology Press.
- Hochreiter, S., & Schmidhuber, J. (1997). Long short-term memory. Neural Computation, 9(8), 1735–1780.
- Güçlü, U., & van Gerven, M. A. (2017). Modeling the dynamics of human brain activity with recurrent neural networks. Frontiers in Computational Neuroscience, 11, 7.
- Graves, A. (2012). Supervised sequence labelling. In Supervised sequence labelling with recurrent neural networks (pp. 5-13). Springer, Berlin, Heidelberg.
- Jarne, C., & Laje, R. (2019). A detailed study of recurrent neural networks used to model tasks in the cerebral cortex. ArXiv Preprint ArXiv:1906.01094.
- John, M. F. (1992). The story gestalt: A model of knowledge-intensive processes in text comprehension. Cognitive Science, 16(2), 271–306.
- Munakata, Y., McClelland, J. L., Johnson, M. H., & Siegler, R. S. (1997). Rethinking infant knowledge: Toward an adaptive process account of successes and failures in object permanence tasks. Psychological Review, 104(4), 686.
- Muñoz-Organero, M., Powell, L., Heller, B., Harpin, V., & Parker, J. (2019). Using Recurrent Neural Networks to Compare Movement Patterns in ADHD and Normally Developing Children Based on Acceleration Signals from the Wrist and Ankle. Sensors (Basel, Switzerland), 19(13). https://doi.org/10.3390/s19132935
- K. J. Lang, A. H. Waibel, and G. E. Hinton. A Time-delay Neural Network Architecture for Isolated Word Recognition. Neural Networks, 3(1):23-43, 1990
- Plaut, D. C., McClelland, J. L., Seidenberg, M. S., & Patterson, K. (1996). Understanding normal and impaired word reading: Computational principles in quasi-regular domains. Psychological Review, 103(1), 56.
- Raj, B. (2020, Spring). Neural Networks: Hopfield Nets and Auto Associators [Lecture]. http://deeplearning.cs.cmu.edu/document/slides/lec17.hopfield.pdf
- Zhang, A., Lipton, Z. C., Li, M., & Smola, A. J. (2020). 8. Recurrent Neural Networks. In Dive into Deep Learning. https://d2l.ai/chapter_convolutional-neural-networks/index.html

## Useful on-line resources